In [27]:
%matplotlib inline
import pandas as pd

In [28]:
import datetime

In [29]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [30]:
import os
# tdir = os.path.join("C:\\trader\journal\_08_August\Week_4\_0821_Tuesday")
# tdir = (r"C:\trader\journal\_08_August\Week_4\_0822_Wednesday")
tdir = (r"C:\trader\journal\_08_August\Week_4\_0823_Thursday")

tdir

'C:\\trader\\journal\\_08_August\\Week_4\\_0823_Thursday'

In [31]:
tfile=os.path.join(tdir, "Trades.csv")

# tfile="TradesWithHolds.csv"
# tfile="TradesExcelEdited.csv"
# tfile="TradesWithBothHolds.csv"

#The copied path from file explorer fails
# t=os.path.join("C:\trader\journal\_08_August\Week_3\_0817_Friday", "Trades.csv")



tfile

'C:\\trader\\journal\\_08_August\\Week_4\\_0823_Thursday\\Trades.csv'

In [32]:
trades = pd.read_csv(tfile)

In [33]:
trades

,Time,Symb,Side,Price,Qty,Route,Broker,Account,Type,Cloid,P / L,Unnamed: 11
0,09:49:53,MU,S,50.19,206,SMRT,NaN,TRIB5621,Margin,AUTO,55.62,NaN
1,10:00:44,MU,S,50.05,619,SMRT,NaN,TRIB5621,Margin,AUTO,80.47,NaN
2,09:49:35,MU,S,50.14,275,SMRT,NaN,TRIB5621,Margin,AUTO,60.50,NaN
3,09:42:23,AMD,B,21.81,924,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
4,15:47:56,MU,B,49.73,500,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
5,09:41:36,AMD,S,21.65,699,SMRT,NaN,TRIB5621,Margin,AUTO,146.79,NaN
6,09:35:29,AMD,B,21.44,932,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
7,09:47:42,MU,B,49.92,1100,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
8,09:40:37,AMD,S,21.72,233,SMRT,NaN,TRIB5621,Margin,AUTO,65.24,NaN
9,09:47:01,AMD,S,21.67,924,SMRT,NaN,TRIB5621,Margin,AUTO,-129.36,NaN


In [34]:
def checkRequiredInputFields(dframe) :
    RequiredFields = ['Time', 'Symb', 'Side', 'Price', 'Qty', 'P / L']
    ActualFields=dframe.columns
    if set (RequiredFields) <= (set (ActualFields)) :
        print("got it")
    else :
        err='You are missing some fields in your input file:\n'
        err += str((set(RequiredFields) - set(ActualFields)))
        raise ValueError(err)


In [35]:
def createDf(dframe, numRow) :
    ''' Creates a new DataFrame with  the length numRow. Each cell is filled with empty string '''

    ll=list()
    r=list()
    for i in range(len(dframe.columns)) :
        r.append('')
        
    for i in range(numRow) :
        ll.append(r)
    newdf= pd.DataFrame(ll, columns=dframe.columns)

    return newdf

In [36]:
def addRows(dframe, numRow) :
    newdf= createDf(dframe, numRow)
        
    
    dframe = dframe.append(newdf, ignore_index=True, sort=False)
    
    return dframe

    

### Make sure the hour string is 0 padded.  Should probably change these to data types. 

### Todo- This, and the duration, will get more complex with the IB report format. The html report uses a long timedate format

In [37]:
def zeroPadTimeStr(dframe, timeHeading) :
    '''Guarantee that the time format xx:xx:xx'''
    

    for i, row in dframe.iterrows():
        tm = row[timeHeading]
        tms=tm.split(":")
        if int(len(tms[0]) < 2) :
            if tms[0].startswith("0") == False :
                tm= "0" + tm
                dframe.at[i, timeHeading] = tm
    return dframe
                

### Todo. Doctor an input csv file to include fractional numer of shares for testing. Make it more modular by checking for 'HOLD'. It might be useful in a windowed version with menus to do things seperately.

In [38]:
def mkShortsNegative(dframe, side, qty) :
    ''' Fix the shares sold to be negative values. Currently assuming DAS values 'B', 'S', 'SS' for 'Side'. 
    '''
    for i, row in dframe.iterrows():
        if row[side] != 'B' and row[qty] > 0:
            dframe.at[i, qty] = ((dframe.at[i, qty]) * -1)
    return dframe

### def getListTickerDF(df) will take a dataframe that includes tickers in column 'Symb' and returns a python list of DataFrames , 1 for each ticker.

In [39]:
def getListTickerDF(dframe, tickCol = 'Symb') :
    ldf_tick = list()
    for ticker in dframe[tickCol].unique() :
        ldf = dframe[dframe[tickCol] == ticker]
        ldf_tick.append(ldf)
    return ldf_tick

In [40]:

def getOvernightTrades(dframe, tickCol='Symb', qtyCol='Qty') :
    ''' getOvernightTrades(dframe) takes the DataFrame and returns a list of lists 
        (symb, qty) of trades that have overnight shares. '''
    
    ldf_tick = getListTickerDF(dframe, tickCol)
    overnightTrade = list()
    for ticker in ldf_tick :
        if ticker[qtyCol].sum() != 0 :
            overnightTrade.append([ticker[tickCol].unique()[0], ticker[qtyCol].sum(), 0, 0])
    return overnightTrade

### Note that this does not yet include those shares that are held before the days trading began. Redo this to remake the list of data frames from the Symbols of Swing List then make a list of data frams that  excludes those then merge them together

In [41]:
def askUser(st, question, ix, default) :
    while True :
        try :
            response = input(question)
            if len(response) < 1 :
                response = default
            else :
                response = int(response)
        except Exception as ex:
            print(ex)
            print("please enter a number")
            continue
       
        st[ix] = response
        return st


In [42]:
# for i in range(len(swingTrade)) :
def figureOvernightTransactions() :
    swingTrade = getOvernightTrades(trades)
    for i in range(len(swingTrade)) :
        tryAgain =  True
        while tryAgain == True :

            xticker = 0      #Candy coaters
            xbalance = 1
            xbefore = 2
            xafter = 3
            print(swingTrade[i])
            print ("There is an unbalanced amount of shares of {0} in the amount of {1}".format(swingTrade[i][xticker], swingTrade[i][xbalance]))

            question = "How many shares of {0} are you holding now? (Enter for {1})".format(swingTrade[i][xticker], swingTrade[i][xbalance])
            swingTrade[i] = askUser(swingTrade[i], question, xafter, swingTrade[i][xbalance])

            if swingTrade[i][xafter] != swingTrade[i][xbalance]:

                difference = swingTrade[i][xbalance] - swingTrade[i][xafter]
                statement = "There is now a prior unbalanced amount of shares of {0} in the amount of {1}"
                print(statement.format(swingTrade[i][xticker], difference))
                question = "How many shares of {0} were you holding before?".format(swingTrade[i][xticker])
                swingTrade[i] = askUser(swingTrade[i], question, xbefore, difference)

            unaccounted = swingTrade[i][xbefore] + swingTrade[i][xafter] - swingTrade[i][xbalance]
            if unaccounted == 0 :
                print("That works.")
                tryAgain = False
            else :
                print()
                print("There are {1} unaccounted for shares in {0}".format(swingTrade[i][xticker], unaccounted))
                print()
                print("That does not add up. Starting over ...")
                print()
                print ("Prior to reset version ", i, swingTrade)
                swingTrade[i] = getOvernightTrades(trades)[i]
                print ("reset version ", i, swingTrade)
    return swingTrade

In [43]:
def insertOvernightRowold(dframe, swingTrade, time='Time', symbol='Symb', side='Side', price='Price', qty='Qty', acct="Account", PL='P / L') :
    newdf = createDf(trades, 0)
    for ldf in getListTickerDF(dframe) :
        found=False
        for ticker, balance, before, after in swingTrade:
            if ticker == ldf.Symb.unique()[0] :
#                 print ("Got {0} with the balance {1}". format (ticker, balance))
                found = True
                ldf = addRows(ldf, 1)
                for i, row in ldf.iterrows():

                    if i == len(ldf) -1 :
                        ldf.at[i, time] = '23:59:59'
                        ldf.at[i, symbol] = ticker
                        if balance > 0 :
                            ldf.at[i, side] = "HOLD+"
                        else :
                            ldf.at[i, side] = "HOLD-"
                        ldf.at[i, price] = 0
                        ldf.at[i, qty] = after
                        ldf.at[i, acct] = 'ZeroSubstance'
                        ldf.at[i, PL] = 0
        newdf = newdf.append(ldf, ignore_index = True)
    return newdf

In [44]:
def insertOvernightRow(dframe, st, time='Time', symbol='Symb', side='Side', price='Price', qty='Qty', acct="Account", PL='P / L') :
    newdf = createDf(trades, 0)
    
    for ldf in getListTickerDF(dframe) :
        found=False
        for ticker, balance, before, after in swingTrade:
            if ticker == ldf.Symb.unique()[0] :
                print ("Got {0} with the balance {1}, before {2} and after {3}". format (ticker, balance, before, after))
                if before != 0 :
                    newldf = createDf(trades, 1)
                    print("length:   ", len(newldf))
                    for i, row in newldf.iterrows():

                        if i == len(newldf) -1 :
                            print("Though this seems unnecessary it will make it more uniform ")
                            newldf.at[i, time] = '00:00:01'
                            newldf.at[i, symbol] = ticker
                            if before > 0 :
                                newldf.at[i, side] = "HOLD+"
                            else :
                                newldf.at[i, side] = "HOLD-"
                            newldf.at[i, price] = 0
                            newldf.at[i, qty] = before
                            newldf.at[i, acct] = 'ZeroSubstance'
                            newldf.at[i, PL] = 0
                            
                            ldf = newldf.append(ldf, ignore_index = True)
                        break #This should be unnecessary as newldf should always be the length of 1 here
                if after != 0 :
                    print("Are we good?")
                    ldf = addRows(ldf, 1)
        
                    for i, row in ldf.iterrows():

                        if i == len(ldf) -1 :
                            ldf.at[i, time] = '23:59:59'
                            ldf.at[i, symbol] = ticker
                            if balance > 0 :
                                ldf.at[i, side] = "HOLD+"
                            else :
                                ldf.at[i, side] = "HOLD-"
                            ldf.at[i, price] = 0
                            ldf.at[i, qty] = after
                            ldf.at[i, acct] = 'ZeroSubstance'
                            ldf.at[i, PL] = 0
        
        newdf = newdf.append(ldf, ignore_index = True, sort = False)
    return newdf

In [45]:
def writeShareBalance(dframe) :
    prevBal = 0
    for i, row in dframe.iterrows():
        qty = (dframe.at[i,'Qty'])
        if row['Side'].startswith("HOLD") :
            print("got it at ", qty)
            qty = qty * -1
        newBalance = qty + prevBal

        dframe.at[i,'Balance'] = newBalance 
        prevBal = newBalance    
    return nt

In [46]:
def addStartTime(dframe) :
    newTrade = True
    for i, row in dframe.iterrows():
#         print(i, row['Time'])
        if newTrade :
            if row['Side'].startswith('HOLD') and i < len(dframe):
                
                oldTime = dframe.at[i+1, 'Time']
#                 print("     :Index: {0},  Side: {1}, Time{2}, setting {3}".format(i, row['Side'], row['Time'], oldTime))
                dframe.at[i, 'Start'] = oldTime

            else :
                oldTime = dframe.at[i, 'Time']
#                 print("     :Index: {0},  Side: {1}, Time{2}, setting {3}".format(i, row['Side'], row['Time'], oldTime))
                dframe.at[i, 'Start'] = oldTime
                
                
            newTrade = False
        else :
#             print("      Finally :Index: {0},  Side: {1}, Time{2}, setting {3}".format(i, row['Side'], row['Time'], oldTime))
            dframe.at[i, 'Start'] = oldTime
        if row['Balance'] == 0 :
#             print("       setting newTrade")
            newTrade = True
    return dframe
        

## Now that we have Trade index and start times everything else in this file should use filters nt[nt.Tindex == "Trade 1"] . This way each trade can be treated as an object that includes a variable number of transactions.

In [47]:
def addTradeIndex(dframe) :

    TCount = 1
    prevEndTrade = -1

    for i, row in dframe.iterrows():
        if len(row['Symb']) < 1 :
            break
        tradeIndex = "Trade " + str(TCount)
        if prevEndTrade == 0:
            TCount = TCount + 1
            prevEndTrade = -1
        tradeIndex = "Trade " + str(TCount)
        dframe.at[i,'Tindex'] = tradeIndex 
        if row['Balance'] == 0 :
            prevEndTrade = 0
    numTrades = TCount
    print(numTrades)        
    return dframe

### This will blow up from wrong types if there is some kind of anomaly. The blank rows in Sum col are str and the filled rows are float. Don't fix it until it blows up. (or you need change this 'now' to production code for some reason) That way we have some case to fix.

###### nt['P / L'].sum()
###### nt['P / L'].dtype

In [48]:
def addTradePL (dframe) :
    tradeTotal = 0.0
    for i, row in dframe.iterrows():
        if row['Balance'] != 0 :
            tradeTotal = tradeTotal + row['P / L']
        else :
            sumtotal = tradeTotal + row['P / L']
            dframe.at[i, 'Sum'] = sumtotal
            tradeTotal = 0
    return dframe


In [49]:
def addTradeDuration(dframe) :
    
    tradeTotal = 0.0
    for i, row in dframe.iterrows():
        if row['Balance'] == 0 :
            timeEnd = row['Time']
            timeStart = row['Start']
            end=timeEnd.split(":")
            start=timeStart.split(":")
            diff = datetime.datetime(1,1,1,int(end[0]), int(end[1]), int(end[2])) - datetime.datetime(1,1,1,int(start[0]), int(start[1]), int(start[2]))
            dframe.at[i,'Duration'] = diff
    return dframe


In [50]:
def addTradeName(dframe) :
    for i, row in dframe.iterrows():
        longShort = " Long"
        if row['Balance'] == 0 :
            if row['Side'] == 'B' :
                longShort = " Short"
            dframe.at[i, 'Name'] = row['Symb'] + longShort
    return dframe



In [51]:
# Note that .sum() should work on this but it failed when I tried it.
def addSummaryPL(dframe) :
    
    count=0
    tot=0.0
    tot2 = 0.0
    for i, row in dframe.iterrows():
        count=count+1
        if count < len(dframe) :
            tot=tot+row['P / L']
            if row['Balance'] == 0 :
 
                tot2 = tot2 + row['Sum']
            if count == len(dframe) -1 :
                lastCol = row['P / L']

                print("Last col?", row['P / L'])
   
        else :
            dframe.at[i, 'P / L'] = tot
            dframe.at[i, 'Sum'] = tot2

    if lastCol > 0:

        print('''
        Some shares are unaccounted for. Please send the original csv file to the developer in 
        order to fix ths issue in the software. Please remove the account number  or change its value
        to anything else.
        ''')
    return dframe


In [52]:
checkRequiredInputFields(trades)

got it


In [53]:
trades = zeroPadTimeStr(trades, 'Time')

In [54]:
trades

,Time,Symb,Side,Price,Qty,Route,Broker,Account,Type,Cloid,P / L,Unnamed: 11
0,09:49:53,MU,S,50.19,206,SMRT,NaN,TRIB5621,Margin,AUTO,55.62,NaN
1,10:00:44,MU,S,50.05,619,SMRT,NaN,TRIB5621,Margin,AUTO,80.47,NaN
2,09:49:35,MU,S,50.14,275,SMRT,NaN,TRIB5621,Margin,AUTO,60.50,NaN
3,09:42:23,AMD,B,21.81,924,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
4,15:47:56,MU,B,49.73,500,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
5,09:41:36,AMD,S,21.65,699,SMRT,NaN,TRIB5621,Margin,AUTO,146.79,NaN
6,09:35:29,AMD,B,21.44,932,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
7,09:47:42,MU,B,49.92,1100,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
8,09:40:37,AMD,S,21.72,233,SMRT,NaN,TRIB5621,Margin,AUTO,65.24,NaN
9,09:47:01,AMD,S,21.67,924,SMRT,NaN,TRIB5621,Margin,AUTO,-129.36,NaN


In [55]:
trades = trades.sort_values(['Symb', 'Time'])

In [56]:
trades

,Time,Symb,Side,Price,Qty,Route,Broker,Account,Type,Cloid,P / L,Unnamed: 11
6,09:35:29,AMD,B,21.44,932,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
8,09:40:37,AMD,S,21.72,233,SMRT,NaN,TRIB5621,Margin,AUTO,65.24,NaN
5,09:41:36,AMD,S,21.65,699,SMRT,NaN,TRIB5621,Margin,AUTO,146.79,NaN
3,09:42:23,AMD,B,21.81,924,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
9,09:47:01,AMD,S,21.67,924,SMRT,NaN,TRIB5621,Margin,AUTO,-129.36,NaN
7,09:47:42,MU,B,49.92,1100,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
2,09:49:35,MU,S,50.14,275,SMRT,NaN,TRIB5621,Margin,AUTO,60.50,NaN
0,09:49:53,MU,S,50.19,206,SMRT,NaN,TRIB5621,Margin,AUTO,55.62,NaN
1,10:00:44,MU,S,50.05,619,SMRT,NaN,TRIB5621,Margin,AUTO,80.47,NaN
4,15:47:56,MU,B,49.73,500,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN


In [57]:
trades = mkShortsNegative(trades, 'Side', 'Qty')

In [58]:
trades

,Time,Symb,Side,Price,Qty,Route,Broker,Account,Type,Cloid,P / L,Unnamed: 11
6,09:35:29,AMD,B,21.44,932,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
8,09:40:37,AMD,S,21.72,-233,SMRT,NaN,TRIB5621,Margin,AUTO,65.24,NaN
5,09:41:36,AMD,S,21.65,-699,SMRT,NaN,TRIB5621,Margin,AUTO,146.79,NaN
3,09:42:23,AMD,B,21.81,924,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
9,09:47:01,AMD,S,21.67,-924,SMRT,NaN,TRIB5621,Margin,AUTO,-129.36,NaN
7,09:47:42,MU,B,49.92,1100,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
2,09:49:35,MU,S,50.14,-275,SMRT,NaN,TRIB5621,Margin,AUTO,60.50,NaN
0,09:49:53,MU,S,50.19,-206,SMRT,NaN,TRIB5621,Margin,AUTO,55.62,NaN
1,10:00:44,MU,S,50.05,-619,SMRT,NaN,TRIB5621,Margin,AUTO,80.47,NaN
4,15:47:56,MU,B,49.73,500,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN


In [59]:
swingTrade = getOvernightTrades(trades)

In [60]:
swingTrade

[['MU', 500, 0, 0]]

In [61]:
swingTrade = figureOvernightTransactions()

['MU', 500, 0, 0]
There is an unbalanced amount of shares of MU in the amount of 500
How many shares of MU are you holding now? (Enter for 500)
That works.


In [63]:
swingTrade

[['MU', 500, 0, 500]]

In [64]:
listdf = insertOvernightRow(trades, swingTrade)

Got MU with the balance 500, before 0 and after 500
Are we good?


In [65]:
listdf

,Time,Symb,Side,Price,Qty,Route,Broker,Account,Type,Cloid,P / L,Unnamed: 11
0,09:35:29,AMD,B,21.44,932,SMRT,NaN,TRIB5621,Margin,AUTO,0,NaN
1,09:40:37,AMD,S,21.72,-233,SMRT,NaN,TRIB5621,Margin,AUTO,65.24,NaN
2,09:41:36,AMD,S,21.65,-699,SMRT,NaN,TRIB5621,Margin,AUTO,146.79,NaN
3,09:42:23,AMD,B,21.81,924,SMRT,NaN,TRIB5621,Margin,AUTO,0,NaN
4,09:47:01,AMD,S,21.67,-924,SMRT,NaN,TRIB5621,Margin,AUTO,-129.36,NaN
5,09:47:42,MU,B,49.92,1100,SMRT,NaN,TRIB5621,Margin,AUTO,0,NaN
6,09:49:35,MU,S,50.14,-275,SMRT,NaN,TRIB5621,Margin,AUTO,60.5,NaN
7,09:49:53,MU,S,50.19,-206,SMRT,NaN,TRIB5621,Margin,AUTO,55.62,NaN
8,10:00:44,MU,S,50.05,-619,SMRT,NaN,TRIB5621,Margin,AUTO,80.47,NaN
9,15:47:56,MU,B,49.73,500,SMRT,NaN,TRIB5621,Margin,AUTO,0,NaN


In [66]:
lbls = ['Tindex', 'Start', 'Time', 'Symb', 'Side', 'Price', 'Qty','Balance', 'Account', "P / L", 'Sum', 'Duration', 'Name']

In [67]:
for l in lbls :
    if l not in listdf.columns :
        listdf[l] = ''
# trades.columns
newTrades = listdf[lbls]
newTrades.copy()
# newTrades.columns

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
0,,,09:35:29,AMD,B,21.44,932,,TRIB5621,0,,,
1,,,09:40:37,AMD,S,21.72,-233,,TRIB5621,65.24,,,
2,,,09:41:36,AMD,S,21.65,-699,,TRIB5621,146.79,,,
3,,,09:42:23,AMD,B,21.81,924,,TRIB5621,0,,,
4,,,09:47:01,AMD,S,21.67,-924,,TRIB5621,-129.36,,,
5,,,09:47:42,MU,B,49.92,1100,,TRIB5621,0,,,
6,,,09:49:35,MU,S,50.14,-275,,TRIB5621,60.5,,,
7,,,09:49:53,MU,S,50.19,-206,,TRIB5621,55.62,,,
8,,,10:00:44,MU,S,50.05,-619,,TRIB5621,80.47,,,
9,,,15:47:56,MU,B,49.73,500,,TRIB5621,0,,,


In [68]:
nt = newTrades.sort_values(['Symb', 'Time'])

In [69]:
nt

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
0,,,09:35:29,AMD,B,21.44,932,,TRIB5621,0,,,
1,,,09:40:37,AMD,S,21.72,-233,,TRIB5621,65.24,,,
2,,,09:41:36,AMD,S,21.65,-699,,TRIB5621,146.79,,,
3,,,09:42:23,AMD,B,21.81,924,,TRIB5621,0,,,
4,,,09:47:01,AMD,S,21.67,-924,,TRIB5621,-129.36,,,
5,,,09:47:42,MU,B,49.92,1100,,TRIB5621,0,,,
6,,,09:49:35,MU,S,50.14,-275,,TRIB5621,60.5,,,
7,,,09:49:53,MU,S,50.19,-206,,TRIB5621,55.62,,,
8,,,10:00:44,MU,S,50.05,-619,,TRIB5621,80.47,,,
9,,,15:47:56,MU,B,49.73,500,,TRIB5621,0,,,


In [70]:
nt = writeShareBalance(nt)

got it at  500


In [71]:
nt

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
0,,,09:35:29,AMD,B,21.44,932,932,TRIB5621,0,,,
1,,,09:40:37,AMD,S,21.72,-233,699,TRIB5621,65.24,,,
2,,,09:41:36,AMD,S,21.65,-699,0,TRIB5621,146.79,,,
3,,,09:42:23,AMD,B,21.81,924,924,TRIB5621,0,,,
4,,,09:47:01,AMD,S,21.67,-924,0,TRIB5621,-129.36,,,
5,,,09:47:42,MU,B,49.92,1100,1100,TRIB5621,0,,,
6,,,09:49:35,MU,S,50.14,-275,825,TRIB5621,60.5,,,
7,,,09:49:53,MU,S,50.19,-206,619,TRIB5621,55.62,,,
8,,,10:00:44,MU,S,50.05,-619,0,TRIB5621,80.47,,,
9,,,15:47:56,MU,B,49.73,500,500,TRIB5621,0,,,


In [72]:
nt = addStartTime(nt)

In [73]:
nt

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
0,,09:35:29,09:35:29,AMD,B,21.44,932,932,TRIB5621,0,,,
1,,09:35:29,09:40:37,AMD,S,21.72,-233,699,TRIB5621,65.24,,,
2,,09:35:29,09:41:36,AMD,S,21.65,-699,0,TRIB5621,146.79,,,
3,,09:42:23,09:42:23,AMD,B,21.81,924,924,TRIB5621,0,,,
4,,09:42:23,09:47:01,AMD,S,21.67,-924,0,TRIB5621,-129.36,,,
5,,09:47:42,09:47:42,MU,B,49.92,1100,1100,TRIB5621,0,,,
6,,09:47:42,09:49:35,MU,S,50.14,-275,825,TRIB5621,60.5,,,
7,,09:47:42,09:49:53,MU,S,50.19,-206,619,TRIB5621,55.62,,,
8,,09:47:42,10:00:44,MU,S,50.05,-619,0,TRIB5621,80.47,,,
9,,15:47:56,15:47:56,MU,B,49.73,500,500,TRIB5621,0,,,


In [74]:
nt = nt.sort_values(['Start', 'Time'])

In [75]:
nt

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
0,,09:35:29,09:35:29,AMD,B,21.44,932,932,TRIB5621,0,,,
1,,09:35:29,09:40:37,AMD,S,21.72,-233,699,TRIB5621,65.24,,,
2,,09:35:29,09:41:36,AMD,S,21.65,-699,0,TRIB5621,146.79,,,
3,,09:42:23,09:42:23,AMD,B,21.81,924,924,TRIB5621,0,,,
4,,09:42:23,09:47:01,AMD,S,21.67,-924,0,TRIB5621,-129.36,,,
5,,09:47:42,09:47:42,MU,B,49.92,1100,1100,TRIB5621,0,,,
6,,09:47:42,09:49:35,MU,S,50.14,-275,825,TRIB5621,60.5,,,
7,,09:47:42,09:49:53,MU,S,50.19,-206,619,TRIB5621,55.62,,,
8,,09:47:42,10:00:44,MU,S,50.05,-619,0,TRIB5621,80.47,,,
9,,15:47:56,15:47:56,MU,B,49.73,500,500,TRIB5621,0,,,


In [76]:
nt = addTradeIndex(nt)

4


In [80]:
nt

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
0,Trade 1,09:35:29,09:35:29,AMD,B,21.44,932,932,TRIB5621,0,,,
1,Trade 1,09:35:29,09:40:37,AMD,S,21.72,-233,699,TRIB5621,65.24,,,
2,Trade 1,09:35:29,09:41:36,AMD,S,21.65,-699,0,TRIB5621,146.79,212.03,,
3,Trade 2,09:42:23,09:42:23,AMD,B,21.81,924,924,TRIB5621,0,,,
4,Trade 2,09:42:23,09:47:01,AMD,S,21.67,-924,0,TRIB5621,-129.36,-129.36,,
5,Trade 3,09:47:42,09:47:42,MU,B,49.92,1100,1100,TRIB5621,0,,,
6,Trade 3,09:47:42,09:49:35,MU,S,50.14,-275,825,TRIB5621,60.5,,,
7,Trade 3,09:47:42,09:49:53,MU,S,50.19,-206,619,TRIB5621,55.62,,,
8,Trade 3,09:47:42,10:00:44,MU,S,50.05,-619,0,TRIB5621,80.47,196.59,,
9,Trade 4,15:47:56,15:47:56,MU,B,49.73,500,500,TRIB5621,0,,,


In [81]:
nt = addTradePL(nt)

In [82]:
nt

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
0,Trade 1,09:35:29,09:35:29,AMD,B,21.44,932,932,TRIB5621,0,,,
1,Trade 1,09:35:29,09:40:37,AMD,S,21.72,-233,699,TRIB5621,65.24,,,
2,Trade 1,09:35:29,09:41:36,AMD,S,21.65,-699,0,TRIB5621,146.79,212.03,,
3,Trade 2,09:42:23,09:42:23,AMD,B,21.81,924,924,TRIB5621,0,,,
4,Trade 2,09:42:23,09:47:01,AMD,S,21.67,-924,0,TRIB5621,-129.36,-129.36,,
5,Trade 3,09:47:42,09:47:42,MU,B,49.92,1100,1100,TRIB5621,0,,,
6,Trade 3,09:47:42,09:49:35,MU,S,50.14,-275,825,TRIB5621,60.5,,,
7,Trade 3,09:47:42,09:49:53,MU,S,50.19,-206,619,TRIB5621,55.62,,,
8,Trade 3,09:47:42,10:00:44,MU,S,50.05,-619,0,TRIB5621,80.47,196.59,,
9,Trade 4,15:47:56,15:47:56,MU,B,49.73,500,500,TRIB5621,0,,,


In [83]:
nt = addTradeDuration(nt)

In [84]:
nt

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
0,Trade 1,09:35:29,09:35:29,AMD,B,21.44,932,932,TRIB5621,0,,,
1,Trade 1,09:35:29,09:40:37,AMD,S,21.72,-233,699,TRIB5621,65.24,,,
2,Trade 1,09:35:29,09:41:36,AMD,S,21.65,-699,0,TRIB5621,146.79,212.03,0:06:07,
3,Trade 2,09:42:23,09:42:23,AMD,B,21.81,924,924,TRIB5621,0,,,
4,Trade 2,09:42:23,09:47:01,AMD,S,21.67,-924,0,TRIB5621,-129.36,-129.36,0:04:38,
5,Trade 3,09:47:42,09:47:42,MU,B,49.92,1100,1100,TRIB5621,0,,,
6,Trade 3,09:47:42,09:49:35,MU,S,50.14,-275,825,TRIB5621,60.5,,,
7,Trade 3,09:47:42,09:49:53,MU,S,50.19,-206,619,TRIB5621,55.62,,,
8,Trade 3,09:47:42,10:00:44,MU,S,50.05,-619,0,TRIB5621,80.47,196.59,0:13:02,
9,Trade 4,15:47:56,15:47:56,MU,B,49.73,500,500,TRIB5621,0,,,


In [85]:
nt = addTradeName(nt)

In [86]:
nt

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
0,Trade 1,09:35:29,09:35:29,AMD,B,21.44,932,932,TRIB5621,0,,,
1,Trade 1,09:35:29,09:40:37,AMD,S,21.72,-233,699,TRIB5621,65.24,,,
2,Trade 1,09:35:29,09:41:36,AMD,S,21.65,-699,0,TRIB5621,146.79,212.03,0:06:07,AMD Long
3,Trade 2,09:42:23,09:42:23,AMD,B,21.81,924,924,TRIB5621,0,,,
4,Trade 2,09:42:23,09:47:01,AMD,S,21.67,-924,0,TRIB5621,-129.36,-129.36,0:04:38,AMD Long
5,Trade 3,09:47:42,09:47:42,MU,B,49.92,1100,1100,TRIB5621,0,,,
6,Trade 3,09:47:42,09:49:35,MU,S,50.14,-275,825,TRIB5621,60.5,,,
7,Trade 3,09:47:42,09:49:53,MU,S,50.19,-206,619,TRIB5621,55.62,,,
8,Trade 3,09:47:42,10:00:44,MU,S,50.05,-619,0,TRIB5621,80.47,196.59,0:13:02,MU Long
9,Trade 4,15:47:56,15:47:56,MU,B,49.73,500,500,TRIB5621,0,,,


In [87]:
nt=addRows(nt,1)

In [88]:
nt = addSummaryPL(nt)
# nt = addSummaryPL(nt)

Last col? 0


In [89]:
nt

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
0,Trade 1,09:35:29,09:35:29,AMD,B,21.44,932,932,TRIB5621,0,,,
1,Trade 1,09:35:29,09:40:37,AMD,S,21.72,-233,699,TRIB5621,65.24,,,
2,Trade 1,09:35:29,09:41:36,AMD,S,21.65,-699,0,TRIB5621,146.79,212.03,0:06:07,AMD Long
3,Trade 2,09:42:23,09:42:23,AMD,B,21.81,924,924,TRIB5621,0,,,
4,Trade 2,09:42:23,09:47:01,AMD,S,21.67,-924,0,TRIB5621,-129.36,-129.36,0:04:38,AMD Long
5,Trade 3,09:47:42,09:47:42,MU,B,49.92,1100,1100,TRIB5621,0,,,
6,Trade 3,09:47:42,09:49:35,MU,S,50.14,-275,825,TRIB5621,60.5,,,
7,Trade 3,09:47:42,09:49:53,MU,S,50.19,-206,619,TRIB5621,55.62,,,
8,Trade 3,09:47:42,10:00:44,MU,S,50.05,-619,0,TRIB5621,80.47,196.59,0:13:02,MU Long
9,Trade 4,15:47:56,15:47:56,MU,B,49.73,500,500,TRIB5621,0,,,


In [90]:
# Now  we are going to add each trade and insert space to put in pictures with circles and arrows and 
# paragraph on the back of each one to be used as evidence against you in a court of law (or court of bb opionion)
insertsize=25
dframe = nt
ldf = list()
count = 1
while True :
    tradeStr = "Trade " + str(count)
    count = count + 1
    tdf = dframe[dframe.Tindex == tradeStr]
    if len(tdf) > 0 :
        ldf.append(tdf)
    else :
        break
len(ldf)

dframe = addRows(dframe, 2)

for tdf in ldf :
    dframe = dframe.append(tdf, ignore_index = True)
    dframe = addRows(dframe, insertsize)
    print(count, ": ", len(dframe))
    
nt = dframe

6 :  42
6 :  69
6 :  98
6 :  125


#####  TODO - Catch Permission Denied exception and inform the user

In [91]:
name=datetime.date.today().strftime("Trades_%A_%m%d.csv")
namexl=datetime.date.today().strftime("Trades_%A_%m%d.xlsx")

# nt.to_csv(name, index=False)
nt.to_excel(namexl, index=False)

In [92]:
# tdir='C:\\trader\\journal\\_08_August\\Week_4\\_0821_Tuesday\\'
# outfile=tdir+ name
# nt.to_csv(outfile)


In [93]:
jname=os.path.join(tdir,namexl)
jname

'C:\\trader\\journal\\_08_August\\Week_4\\_0823_Thursday\\Trades_Thursday_0823.xlsx'

In [94]:
nt.to_excel(jname, index=False)